Load Libraries

In [1]:
from dotenv import load_dotenv
import os
from math import ceil
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
load_dotenv()

/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

Load LLM Model

In [2]:
quantization_config = BitsAndBytesConfig(load_in_8bit = True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct",
                                             quantization_config = quantization_config, 
                                             token = os.getenv("HUGGINGFACE_TOKEN"))
model.config.use_cache = False
model.config.pretraining_tp = 1

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("pcuenq/Llama-3.2-1B-Instruct-tokenizer", 
                                          trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

Load Dataset

In [4]:
dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train")
dataset

Dataset({
    features: ['text'],
    num_rows: 6861
})

Training Args

In [5]:
sample_size = dataset.num_rows
batch_size = 4

In [6]:
args = TrainingArguments(output_dir = "./llama_finetune", 
                         per_device_train_batch_size = batch_size, 
                         max_steps = 5 * (ceil(sample_size/batch_size)))

Supervised Fine-Tuning

In [7]:
trainer = SFTTrainer(model = model, args = args, 
                     train_dataset = dataset,
                     tokenizer = tokenizer, 
                     peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 128, lora_alpha = 16, lora_dropout = .1, ),
                     dataset_text_field = "text")

/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hawkiyc. Use `wandb login --relogin` to force relogin


  0%|          | 0/8580 [00:00<?, ?it/s]/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  6%|▌         | 500/8580 [11:29<3:12:31,  1.43s/it]

{'loss': 1.8533, 'grad_norm': 0.13843896985054016, 'learning_rate': 4.708624708624709e-05, 'epoch': 0.29}


/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67123521-330efa8f784928e91f009c6f;d10c12e3-bf19-4427-bcf6-c52a579c525d)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B-Instruct.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarn

{'loss': 1.7424, 'grad_norm': 0.17795269191265106, 'learning_rate': 4.4172494172494175e-05, 'epoch': 0.58}


...

/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67125dde-3de5f327019f145362e20358;69f96d1d-abb0-40f6-a89d-33767f68acaa)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B-Instruct.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarn

{'loss': 1.7083, 'grad_norm': 0.18375183641910553, 'learning_rate': 4.662004662004662e-07, 'epoch': 4.95}


/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67126095-28fda80c782419166e4e62fc;ab082d27-0480-4362-b026-9a39178f0cd4)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B-Instruct.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarn

{'train_runtime': 11929.895, 'train_samples_per_second': 2.877, 'train_steps_per_second': 0.719, 'train_loss': 1.725535412768384, 'epoch': 5.0}


TrainOutput(global_step=8580, training_loss=1.725535412768384, metrics={'train_runtime': 11929.895, 'train_samples_per_second': 2.877, 'train_steps_per_second': 0.719, 'total_flos': 2.01700558334976e+17, 'train_loss': 1.725535412768384, 'epoch': 5.0})

In [9]:
prompt = "What is malaria?"
text_generation_pipeline = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {prompt} [/INST]")
print(model_answer[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/revlis_ai/anaconda3/envs/env_llm/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s>[INST] What is malaria? [/INST] Malaria is a serious and sometimes life-threatening disease caused by a parasite of the Plasmodium genus. The parasite infects red blood cells and causes the red blood cells to die. The disease is spread by the bite of an infected female Anopheles mosquito. Symptoms include fever, chills, flu-like symptoms, and in severe cases, a headache, vomiting, and in some cases, kidney failure and death. Malaria is a major public health problem in tropical and subtropical regions of the world, with an estimated 228 million cases and 435,000 deaths in 2019.
The disease is caused by the Plasmodium species P. vivax, P. ovale, P. malariae, and P. falciparum, which are all transmitted by the Anopheles mosquito. The parasite is transmitted through the bite of an infected female Anopheles mosquito that feeds on the blood of an infected individual, usually after 10 days of infection. The parasite multiplies in the red blood cells of the host, and the infected red blood 

In [10]:
user_prompt = "Please tell me about Bursitis"
text_generation_pipeline = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

<s>[INST] Please tell me about Bursitis [/INST] Bursitis is inflammation of a bursa, a fluid-filled sac that cushions a joint, tendon, or muscle. Bursitis is often caused by repetitive motion, injury, or infection. It can be a chronic or acute condition. Bursitis is usually painless and can be associated with other conditions such as arthritis. Bursitis is a common cause of joint pain and can be treated with medication, physical therapy, or surgery.
Inflammation of the bursa, the bursa is a fluid-filled sac that cushions a joint, tendon, or muscle. The bursa is filled with synovial fluid, a fluid that is produced by the synovial membrane. The bursa is located in the area of the joint, tendon, or muscle that is subjected to repetitive motion. The bursa is a protective layer that reduces friction between the joint, tendon, or muscle and the bone. When the bursa becomes inflamed, the friction between the joint, tendon, or muscle and the bone increases, causing pain and inflammation.
Bursi

wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
